In [15]:
import spacy 
import random 
from spacy.util import minibatch,compounding 
from pathlib import Path 
from spacy import displacy 
import re 
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from pathlib import Path 

In [2]:
nlp=spacy.load('en_core_web_sm')
ner=nlp.get_pipe("ner")

In [3]:
train=[("""The initial temperature of a rod 0 < x < l is an arbitrary function of x. The temperatures of the ends are
constant: u(0,t)=U1=const, u(l,t)=U2=const, 0 < t < inf. A heat exchange obeying Newton’s law takes place at
the surface with a medium whose temperature equals u0 = const. Find the temperature of the rod. Consider,
in particular, the case where U1=U2 =0.""",{"entities":[(134,143,"boundary_condition"),(117,126,"boundary_condition"),(49,73,"initial_condition")]}),
("""Determine the temperature of a rod 0 < x < l thermally insulated along its surface, if one of its
ends (x = 0) is kept at a given fixed temperature, and a given constant heat flow is maintained at
the other end (x = l), the initial temperature being arbitrary.""",{"entities":[(130,147,"boundary_condition"),(161,179,"boundary_condition"),(250,259,"initial_condition")]}),
("""Find the temperature of a rod 0 < x < l thermally insulated along the surface, if heat sources of
density equal to Φ(t) sin(πx/l) are continuously distributed over the rod, and the initial temperature
of the rod is an arbitrary function f(x) and the temperature of the ends is maintained equal to
zero.""",{"entities":[(297,301,"boundary_condition"),(218,236,"initial_condition")]}),
("""Find the distribution of temperature in a rod 0 ≤ x ≤ l thermally insulated along the surface, if
the temperature of its ends is maintained to zero, and the initial temperature equals an arbitrary
function f(x).""",{"entities":[(143,147,"boundary_condition"),(187,205,"initial_condition")]}),
(""" Find the distribution of temperature in a rod 0 ≤ x ≤ l thermally insulated along the surface, if
the temperature of its ends is maintained to zero, and the initial temperature equals is constant
U0""",{"entities":[(144,148,"boundary_condition"),(197,199,"initial_condition")]}),
("""The initial temperature of a rod 0 ≤ x ≤ l thermally insulated along the surface equals U0,
and a constant temperature is maintained at its ends u(0,t)=U1=const ; u(l,t)=U2=const ; 0 < t < inf :
Find the temperature u(x,t) of the rod for t > 0""",{"entities":[(145,154,"boundary_condition"),(163,172,"boundary_condition"),(88,90,"initial_condition")]}),
("""Find the distribution of temperature in a rod of length l thermally insulated along the surface, if
the temperature of its one end (x = 0) is maintained at constant value U1, and the temperature of
another end (x = l) is zero, and the initial temperature equals an arbitrary function Φ(x).""",{"entities":[(171,173,"boundary_condition"),(221,225,"boundary_condition"),(265,283,"initial_condition")]}),
("""Find the distribution of temperature in a rod of length L0 thermally insulated along the surface, if
the temperature of its one end (x = 0) is zero, and the temperature of another end (x = L0) is constant U0, and the initial temperature is zero.""",
{"entities":[(143,147,"boundary_condition"),(205,207,"boundary_condition"),(240,244,"initial_condition")]}),
("""One end of a rod is fixed elastically, and the other end is free. Find the longitudinal vibrations of the rod for arbitrary initial conditions.""",
{"entities":[(20,36,"boundary_condition"),(60,64,"boundary_condition"),(114,123,"initial_condition")]}),
("""Find the longitudinal vibrations of a rod with elastically fixed ends with identical coefficients of rigidity, if the initial conditions are arbitrary.""",
{"entities":[(59,64,"boundary_condition"),(141,150,"initial_condition")]}),
("""Find the longitudinal vibrations of a rod with elastically fixed ends with different coefficients of rigidity, if the initial conditions are arbitrary.""",
{"entities":[(59,64,"boundary_condition"),(141,150,"initial_condition")]}),
("""Investigate the transverse vibration of a rod for arbitrary initial conditions, if the ends of the rod are fixed by hinges.""",
{"entities":[(107,112,"boundary_condition"),(50,59,"initial_condition")]}),
("""Investigate the transverse vibration of a rod for arbitrary initial conditions, if the ends of the rod are rigidly fixed.""",
{"entities":[(115,120,"boundary_condition"),(50,59,"initial_condition")]}),
("""Investigate the transverse vibration of a rod for arbitrary initial conditions, if the ends of the rod are free.""",
{"entities":[(107,111,"boundary_condition"),(50,59,"initial_condition")]}),
("""Find the longitudinal vibrations of a rod, one end of which (x = 0) is fixed, and the other (x = L0) is free, for the initial conditions u(x,0)=kx; @u/@t(x,0)=0; for 0<x<L0:""",
{"entities":[(71,76,"boundary_condition"),(104,108,"boundary_condition"),(137,146,"initial_condition"),(148,160,"initial_condition")]}),
("""Find the longitudinal vibrations of a flexible rod, with free ends, if the initial velocities and initial displacements in a longitudinal direction are arbitrary.""",
{"entities":[(57,61,"boundary_condition"),(152,161,"initial_condition")]}),
("""A string 0 < x < L0 with fixed ends, up to the time t = 0, was in a state of equilibrium under the action of a transverse force F0 = const, applied at the point x0 of the string, perpendicularly to the undisturbed position of the string. At the initial time t = 0 the action of the force F0 ceases instantaneously. Find the vibration of the string for t > 0.""",
{"entities":[(25,30,"boundary_condition"),(288,297,"initial_condition")]}),
("""The ends of a string are fixed, and the initial deflection has the form of a quadratic parabola, symmetrical with respect to the perpendicular to the mid-point of the string. Find the vibrations of the string, if the initial velocities equal zero.""",
{"entities":[(25,30,"boundary_condition"),(242,246,"initial_condition")]}),
]

In [6]:
for _,annotations in train:
    for ent in annotations.get("entities"):
        ner.add_label(ent[2])
        
disable_pipes=[pipe for pipe in nlp.pipe_names if pipe !='ner']
def training(train,interation,nlp):
    
    with nlp.disable_pipes(*disable_pipes):
        optimizer=nlp.begin_training()
        for iteration in range(100):
            random.shuffle(train)
            losses={}
            batches=minibatch(train,size=compounding(1.0 ,4.0 ,1.001))
            for batch in batches:
                texts,annotations=zip(*batch)
                nlp.update(
                            texts,
                            annotations,
                            drop=0.2,
                            losses=losses,
                            sgd=optimizer)
                print("losses",losses)

In [7]:
training(train,100,nlp)

losses {'ner': 22.979326998072775}
losses {'ner': 40.40000146079551}
losses {'ner': 104.10202887737601}
losses {'ner': 138.75121868900771}
losses {'ner': 144.50497995155723}
losses {'ner': 173.361297470968}
losses {'ner': 202.7988848135338}
losses {'ner': 230.96991861967848}
losses {'ner': 235.94198607304827}
losses {'ner': 265.5855479756789}
losses {'ner': 275.6914239348331}


/home/zohan/PycharmProjects/pythonProject/deepmodel/lib/python3.8/site-packages/spacy/language.py:482: UserWarning: [W030] Some entities could not be aligned in the text "One end of a rod is fixed elastically, and the oth..." with entities "[(20, 36, 'boundary_condition'), (60, 64, 'boundar...". Use `spacy.gold.biluo_tags_from_offsets(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities (with BILUO tag '-') will be ignored during training.
  gold = GoldParse(doc, **gold)


losses {'ner': 290.4252612142069}
losses {'ner': 309.00685758219936}
losses {'ner': 318.4533404866585}
losses {'ner': 347.9881225731298}
losses {'ner': 358.87019672484723}
losses {'ner': 367.73140566157065}
losses {'ner': 385.406412792987}
losses {'ner': 24.234326836665787}
losses {'ner': 39.14119539404646}
losses {'ner': 48.93812332214844}
losses {'ner': 68.3542682017555}
losses {'ner': 73.91324164528757}
losses {'ner': 87.25765645072804}
losses {'ner': 104.2485763150149}
losses {'ner': 115.21417093107502}
losses {'ner': 143.87755940143742}
losses {'ner': 150.44848733845558}
losses {'ner': 155.95185115736692}
losses {'ner': 164.2270982858113}
losses {'ner': 171.55730442846428}
losses {'ner': 209.55941353178744}
losses {'ner': 228.85354998835194}
losses {'ner': 242.60024419346146}
losses {'ner': 258.9185550193105}
losses {'ner': 267.08146814077475}
losses {'ner': 16.336504768706163}
losses {'ner': 21.304166872514656}
losses {'ner': 43.652815225819836}
losses {'ner': 63.31596787952924}


losses {'ner': 312.06117105455724}
losses {'ner': 335.64880057037317}
losses {'ner': 8.390649693207934}
losses {'ner': 15.442272620795165}
losses {'ner': 37.86923086686443}
losses {'ner': 50.83421518219552}
losses {'ner': 90.68093035554489}
losses {'ner': 108.78696000481727}
losses {'ner': 120.44514792347331}
losses {'ner': 149.20560794922244}
losses {'ner': 160.73811193395727}
losses {'ner': 188.29592309448913}
losses {'ner': 202.0834897245295}
losses {'ner': 216.03849329426362}
losses {'ner': 243.36167938925269}
losses {'ner': 262.03275234286616}
losses {'ner': 269.0360709921147}
losses {'ner': 276.60610503095904}
losses {'ner': 289.014964472909}
losses {'ner': 307.23195435650905}
losses {'ner': 9.076548658462343}
losses {'ner': 27.638874193571155}
losses {'ner': 64.57795732972363}
losses {'ner': 81.38462394165026}
losses {'ner': 93.34931875203006}
losses {'ner': 100.60396680388902}
losses {'ner': 114.09016507328914}
losses {'ner': 127.27626823491909}
losses {'ner': 151.5368812022777

losses {'ner': 152.25226057462055}
losses {'ner': 180.80138878339562}
losses {'ner': 201.96340901021972}
losses {'ner': 239.3604892038427}
losses {'ner': 250.6289208481801}
losses {'ner': 257.03864216070065}
losses {'ner': 276.8548388142706}
losses {'ner': 297.69762019250334}
losses {'ner': 309.4485270203975}
losses {'ner': 317.7361666886279}
losses {'ner': 325.04236777478286}
losses {'ner': 348.82816284681826}
losses {'ner': 353.7132277539788}
losses {'ner': 6.173442257582792}
losses {'ner': 42.80803716287119}
losses {'ner': 55.08017943761297}
losses {'ner': 66.95790649299488}
losses {'ner': 85.80901586768096}
losses {'ner': 94.73028891813362}
losses {'ner': 114.58924629006322}
losses {'ner': 120.22098909313752}
losses {'ner': 137.2931566625524}
losses {'ner': 143.32206770466325}
losses {'ner': 169.7726485321594}
losses {'ner': 177.7399134611672}
losses {'ner': 201.9706100029503}
losses {'ner': 221.14257605608105}
losses {'ner': 248.2270734886984}
losses {'ner': 253.53570040920005}
lo

losses {'ner': 185.83863552694703}
losses {'ner': 198.02014331800564}
losses {'ner': 219.80024327748762}
losses {'ner': 253.03711343058168}
losses {'ner': 259.0762615751072}
losses {'ner': 14.378116719273748}
losses {'ner': 36.63004804898344}
losses {'ner': 51.19001236480506}
losses {'ner': 62.40568476035898}
losses {'ner': 69.22981878527673}
losses {'ner': 94.15581689837794}
losses {'ner': 99.0938429043328}
losses {'ner': 103.83464126832922}
losses {'ner': 110.8377989780326}
losses {'ner': 129.9475640202715}
losses {'ner': 135.9842786303547}
losses {'ner': 160.42445650318967}
losses {'ner': 197.54016227630763}
losses {'ner': 215.10607799354642}
losses {'ner': 234.78621064102177}
losses {'ner': 262.4277213415304}
losses {'ner': 283.4482269571308}
losses {'ner': 321.5269808188141}
losses {'ner': 22.169442670130593}
losses {'ner': 40.51007976330402}
losses {'ner': 60.68355554903792}
losses {'ner': 68.6166562101532}
losses {'ner': 88.68668009688778}
losses {'ner': 98.7383759203826}
losses

losses {'ner': 4.605084299640994}
losses {'ner': 25.82498805732027}
losses {'ner': 48.945945173130866}
losses {'ner': 76.96114021116225}
losses {'ner': 98.46698128137015}
losses {'ner': 119.28096560106425}
losses {'ner': 149.7609998108521}
losses {'ner': 164.6899229966377}
losses {'ner': 177.40868693368722}
losses {'ner': 215.50807779297722}
losses {'ner': 238.881539131072}
losses {'ner': 246.21723609884802}
losses {'ner': 255.90017291807462}
losses {'ner': 271.77135519352225}
losses {'ner': 293.68546042817354}
losses {'ner': 298.56399385163905}
losses {'ner': 307.02320996647114}
losses {'ner': 317.52182097424094}
losses {'ner': 4.663650923151295}
losses {'ner': 16.165231308781927}
losses {'ner': 42.85594095561664}
losses {'ner': 63.58803399725202}
losses {'ner': 96.85282293532708}
losses {'ner': 98.50733423238255}
losses {'ner': 121.06746329167537}
losses {'ner': 141.93940475251532}
losses {'ner': 164.38584599705985}
losses {'ner': 170.76504182974543}
losses {'ner': 179.4830777049889}

losses {'ner': 154.56562095055028}
losses {'ner': 187.14556233096687}
losses {'ner': 203.4809091189038}
losses {'ner': 203.56649234646216}
losses {'ner': 212.66297289011095}
losses {'ner': 262.1629130768499}
losses {'ner': 286.58327325728885}
losses {'ner': 303.7493710474954}
losses {'ner': 330.29199894298097}
losses {'ner': 358.3776978446857}
losses {'ner': 20.575380582868092}
losses {'ner': 49.06812506284314}
losses {'ner': 61.595932508981605}
losses {'ner': 74.8435722784468}
losses {'ner': 106.81085064628655}
losses {'ner': 124.38215883595478}
losses {'ner': 146.0194341279181}
losses {'ner': 161.0911634062586}
losses {'ner': 172.03482814686595}
losses {'ner': 185.64230126113853}
losses {'ner': 222.3674286606298}
losses {'ner': 264.4303879243329}
losses {'ner': 272.13175058518516}
losses {'ner': 290.4370336666897}
losses {'ner': 312.9648585293105}
losses {'ner': 323.6466453608102}
losses {'ner': 338.400940921714}
losses {'ner': 359.0456142012006}
losses {'ner': 11.850846808135685}
lo

losses {'ner': 293.7487123084365}
losses {'ner': 297.50472290734035}
losses {'ner': 17.29601657547755}
losses {'ner': 31.468801669162815}
losses {'ner': 60.063583127322374}
losses {'ner': 80.93354979362971}
losses {'ner': 102.96183757685148}
losses {'ner': 119.01476804838723}
losses {'ner': 123.5025458645322}
losses {'ner': 134.71615911967456}
losses {'ner': 156.7936119000326}
losses {'ner': 195.20179707576904}
losses {'ner': 215.93393192618686}
losses {'ner': 223.9848509250093}
losses {'ner': 240.62600944452987}
losses {'ner': 249.3144089026442}
losses {'ner': 257.84376255365476}
losses {'ner': 283.6949653948335}
losses {'ner': 305.8234841474973}
losses {'ner': 317.54909861191516}
losses {'ner': 24.645766468012127}
losses {'ner': 33.475542577349344}
losses {'ner': 41.947662833025035}
losses {'ner': 68.25630730417429}
losses {'ner': 71.85005480801499}
losses {'ner': 79.50488008214359}
losses {'ner': 85.97910557069501}
losses {'ner': 94.32718097038294}
losses {'ner': 112.79000283924059}

losses {'ner': 131.90671074103696}
losses {'ner': 153.9264880814863}
losses {'ner': 172.19112424811823}
losses {'ner': 176.6793177747932}
losses {'ner': 183.77490247014796}
losses {'ner': 188.4779211093045}
losses {'ner': 202.7567469616789}
losses {'ner': 216.98388477738484}
losses {'ner': 228.46950831741768}
losses {'ner': 237.02297354640066}
losses {'ner': 245.5403529106388}
losses {'ner': 0.9485551038026198}
losses {'ner': 17.687708748929026}
losses {'ner': 31.40009644703286}
losses {'ner': 47.708929659133496}
losses {'ner': 55.44972531502258}
losses {'ner': 61.84949740954897}
losses {'ner': 84.84448030079562}
losses {'ner': 103.23094731597592}
losses {'ner': 116.75418469694961}
losses {'ner': 151.72186437913768}
losses {'ner': 167.14994321097785}
losses {'ner': 188.36285728422718}
losses {'ner': 192.94952659756888}
losses {'ner': 197.26953393821728}
losses {'ner': 220.38039866053273}
losses {'ner': 225.66745127703146}
losses {'ner': 250.48279662782767}
losses {'ner': 263.4177419259

In [10]:
#trained labels#
for text,_ in train:
    doc=nlp(text)
    print('entities',[(ent.text,ent.label_) for ent in doc.ents])

entities [('free', 'boundary_condition'), ('arbitrary', 'initial_condition')]
entities [('U1', 'boundary_condition'), ('zero', 'boundary_condition'), ('arbitrary function', 'initial_condition')]
entities [('arbitrary function', 'initial_condition'), ('u(0,t)=U1', 'boundary_condition'), ('u(l,t)=U2', 'boundary_condition')]
entities [('zero', 'boundary_condition'), ('U0', 'initial_condition')]
entities [('free', 'boundary_condition')]
entities [('zero', 'boundary_condition'), ('arbitrary\n', 'initial_condition')]
entities [('fixed', 'boundary_condition'), ('F0 =', 'initial_condition'), ('F0 ceases', 'initial_condition')]
entities [('fixed', 'boundary_condition'), ('zero', 'initial_condition')]
entities [('zero', 'boundary_condition'), ('U0', 'boundary_condition'), ('zero', 'initial_condition')]
entities [('fixed temperature', 'boundary_condition'), ('constant heat flow', 'boundary_condition'), ('arbitrary', 'initial_condition')]
entities [('arbitrary function', 'initial_condition'), ('ze

In [ ]:
#example#
doc=nlp("""The initial temperature of a rod 0 < x < L0 thermally insulated along the surface equals zero,
and a constant temperature is maintained at its ends u(0,t)=U1=const ; u(L0,t)=U2=const ; 0 < t < 1: Find the temperature u(x; t) of the rod for t > 0.""")   

for ent in doc.ents: 
    print(ent.text,ent.start_char,ent.end_char,ent.label_)
spacy.displacy.serve(doc,style='ent')

In [8]:
text="""Find the longitudinal vibration of a rod 0 < x < L0, the left end of which is fixed, and a force F(t)=At, t < 0 < 1, A = const., is applied the right end at time t = 0, assuming that the medium does not produce a resistance to the vibrations. The initial conditions are homogeneous."""
def reformulate(text):
    
    pattern1=re.compile('F\([a-z]\)=.*?(?=,)')
    pattern2=re.compile('F\([a-z],t\)=F\([a-z]\).*?(?= )')

    match=[re.search(pattern,text).group() for pattern in [pattern1,pattern2] if re.search(pattern,text) is not None]
    #match[0]
    if 'temprature' in text:      #the problem is of parabolic type 
        DFE='du/dt-a*a*d(du/dt)'
        if 'heat source' or "Newton's law" or "Fourier's law"  not in text: #equation is homogenous
            print("diffential equation: "+'du/dt=a*a*d(du/dt)/dt')
        else:
            DFE='non-homogenous' 
        
    elif 'vibration' in text:      #problem type of hyperbolic type
        DF='hyperbolic'
        if 'force' or 'excitaion' not in text:    #problem is homogenous
            print("diffential equation: "+'du/dt=a*a*d(du/dt)/dt')
        else:
            print("diffential equation: "+'du/dt=a*a*d(du/dt)/dt'+match[0])
          
    doc=nlp(text)
    for ent in doc.ents:
        #print(ent.text,ent.label_)
        if ent.label_=='boundary_condition':
            print("boundary condition: "+ent.text)

        if ent.label_=='initial_condition':    
            print("initial condition: "+ent.text)
        else:
            print("no initial condition found")


    
    

In [33]:
text="""Find the longitudinal vibration of a rod 0 < x < L0, the left end of which is fixed, and a force F(t)=At, t < 0 < 1, A = const., is applied the right end at time t = 0, assuming that the medium does not produce a resistance to the vibrations. The initial conditions are homogeneous."""
reformulate(text)

diffential equation: du/dt=a*a*d(du/dt)/dt
boundary condition: fixed
no initial condition found
boundary condition: F(t)=At,
no initial condition found


In [34]:
#evaluation#
nlp2=spacy.load('/home/zohan/PycharmProjects/pythonProject/ner_model')
print('model loaded')
test=[("""The initial temperature of a rod 0 < x < L0 thermally insulated along the surface equals zero,
and a constant temperature is maintained at its ends u(0,t)=U1=const ; u(L0,t)=U2=const ; 0 < t < 1: Find the temperature u(x; t) of the rod for t > 0.""",
{"entities":[(148,157,"boundary_condition"), (166,176,"boundary_condition"),(89,93,"initial_condition")]}),
("""Find the temperature distribution in a rod 0 < x < L0 thermally insulated along the surface, if a temperature, equal to zero, is maintained at its end x = 0 and at the end x = L0 the temperature varies according to the law u(L0,t)=A0t; 0 < t < 1: The initial temperature of the rod equals zero.""",
{"entities":[(120,124,"boundary_condition"),(223,234,"boundary_condition"),(289,293,"initial_condition")]}),
("""Find the longitudinal vibrations of a rod 0 < x < L0, if one of its ends is rigidly fixed, and a force F0 = const. is applied to the other end at time t = 0. The initial displacement and velocity are zero.""",
{"entities":[(84,89,"boundary_condition"),(103,113,"boundary_condition"),(200,204,"initial_condition")]}),
("""A continuously distributed force of linear density F(x,t)=F(x)sin(wt) is applied for t > 0 to a string 0 < x < L0 with fixed ends. Find the vibrations of the string, if the initial displacement and velocity are zero.""",
{"entities":[(119,124,"boundary_condition"),(211,215,"initial_condition")]}),
("""Find the longitudinal vibration of a rod 0 < x < L0, the end x = 0 of which is rigidly fixed, and the end x = L0, starting at time t = 0, moves according to the law u(L0,t)=A*sin*omega*t, t < 0 < 1. The initial conditions are homogeneous. The medium does not produce a resistance to the vibrations.""",
{"entities":[(87,92,"boundary_condition"),(165,186,"boundary_condition"),(226,237,"initial_condition")]}),
("""Find the longitudinal vibration of a rod 0 < x < L0, the left end of which is fixed, and a force F(t)=At, t < 0 < 1, A = const., is applied the right end at time t = 0, assuming that the medium does not produce a resistance to the vibrations. The initial conditions are homogeneous.""",
{"entities":[(78,83,"boundary_condition"),(97,104,"boundary_condition"),(270,281,"initial_condition")]})]
def evalu(examples,ner_model):
    scorer=Scorer()
    for input_,annot in examples:
        doc_gold_text=ner_model.make_doc(input_)
        gold=GoldParse(doc_gold_text,entities=annot['entities'])
        pred_value=ner_model(input_)
        scorer.score(pred_value,gold)
    return scorer.scores
print(evalu(test,nlp2))

model loaded
{'uas': 0.0, 'las': 0.0, 'las_per_type': {'det': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'amod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nsubj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'prep': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'pobj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nummod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'dep': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'acl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'root': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'dobj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'cc': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'nsubjpass': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'auxpass': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'poss': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'compound': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'appos': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'npadvmod': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'ccomp': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'mark': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'advcl': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'conj': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'attr'